# Python ETL Data Crawling

## 使用requests.get

In [1]:
import requests
res = requests.get('https://www.python.org/')
#print res.text
print res.status_code 
print res.headers['content-type']


200
text/html; charset=utf-8


C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


## 使用requests.post

In [3]:
import requests
payload = {
'StartStation':'977abb69-413a-4ccf-a109-0272c24fd490',
'EndStation':'60831846-f0e4-47f6-9b5b-46323ebdcef7',
'SearchDate':'2015/08/29',
'SearchTime':'10:30',
'SearchWay':'DepartureInMandarin'
}
res = requests.post('http://www.thsrc.com.tw/tw/TimeTable/SearchResult', data=payload)
#print res.text


## 台鐵時課表查詢

In [10]:
import requests
res = requests.get('http://twtraffic.tra.gov.tw/twrail/SearchResult.aspx?searchtype=0&searchdate=2015/08/29&fromstation=1810&tostation=1008&trainclass=%271100%27,%271101%27,%271102%27,%271107%27,%271108%27,%271110%27,%271120%27&fromtime=0600&totime=2359')
#print res.text


## 將網頁讀進BeautifulSoup 中

In [4]:
from bs4 import BeautifulSoup 
html_sample = ' \
<html> \
 <body> \
 <h1 id="title">Hello World</h1> \
 <a href="#" class="link">This is link1</a> \
 <a href="# link2" class="link">This is link2</a> \
 </body> \
 </html>'

soup = BeautifulSoup(html_sample)
print soup.text


   Hello World This is link1 This is link2  


## 使用select 找出含有a tag 的元素

In [14]:
soup = BeautifulSoup(html_sample) 
alink = soup.select('a') 
print alink

[<a class="link" href="#">This is link1</a>, <a class="link" href="# link2">This is link2</a>]


## 取得所有a tag 內的連結

In [15]:
alinks = soup.select('a') 
for link in alinks: 
    print link['href']

#
# link2


## 使用select 找出id為title的元素

In [13]:
alink = soup.select('#title') 
print alink

[<h1 id="title">Hello World</h1>]


## 使用select 找出所有class為link的元素

In [7]:
soup = BeautifulSoup(html_sample) 
for link in soup.findAll('a', {'class': 'link'}): 
    print link

<a class="link" href="#">This is link1</a>
<a class="link" href="# link2">This is link2</a>


## 設定POST 的Form Data

In [17]:
# -*- coding: utf-8 -*- 
import requests

payload = { 'method':'search', 'searchMethod':'true', 'searchTarget':'ATM',
 'orgName':'', 'orgId':'', 'hid_1':'1',
 'tenderName':'', 'tenderId':'', 'tenderStatus':'4,5,21,29',
 'tenderWay':'', 'awardAnnounceStartDate':'104/07/24',
 'awardAnnounceEndDate':'104/07/24', 'radProctrgCate':'3', 
 'proctrgCate':'3', 'tenderRange':'', 'minBudget':'',
 'maxBudget':'', 'item':'','hid_2':'1',
 'gottenVendorName':'', 'gottenVendorId':'', 'hid_3':'1',
 'submitVendorName':'', 'submitVendorId':'', 'location':'',
 'priorityCate':'', 'isReConstruct':'', 'btnQuery':'查詢' }


## 送出Post 取得資料

In [18]:
user_post = requests.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
response_text = user_post.text.encode('utf8') 
#print response_text

## 如何延續Session

In [19]:
# -*- coding: utf-8 -*- 
import requests

payload = { 'method':'search', 'searchMethod':'true', 'searchTarget':'ATM',
 'orgName':'', 'orgId':'', 'hid_1':'1',
 'tenderName':'', 'tenderId':'', 'tenderStatus':'4,5,21,29',
 'tenderWay':'', 'awardAnnounceStartDate':'104/07/24',
 'awardAnnounceEndDate':'104/07/24', 'radProctrgCate':'3', 
 'proctrgCate':'3', 'tenderRange':'', 'minBudget':'',
 'maxBudget':'', 'item':'','hid_2':'1',
 'gottenVendorName':'', 'gottenVendorId':'', 'hid_3':'1',
 'submitVendorName':'', 'submitVendorId':'', 'location':'',
 'priorityCate':'', 'isReConstruct':'', 'btnQuery':'查詢' }

In [20]:
rs = requests.session() 
rs_post = rs.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
rs_get = rs.get("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2") 
response = rs_get.text.encode('utf8')

## 取得資料筆數

In [21]:
from bs4 import BeautifulSoup

user_post = requests.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
response_text = user_post.text.encode('utf8') 
soup = BeautifulSoup(response_text) 
rec_number_element = soup.select('.T11b')[0]
rec_number = int(rec_number_element.text) 
print rec_number

214


## 將資料筆數轉換成頁數

In [22]:
from math import ceil 
page_number = int(ceil(float(rec_number) / 100)) 
print page_number


3


## 字串格式化 - 使用format

In [23]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex={0}" 
for page in range(1, page_number + 1): 
    print page_format.format(page)

http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=1
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=3


## 使用%

In [24]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    print page_format%(page)

http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=1
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2
http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=3


## 取得每頁標案清單的內容

In [25]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    bid_list = rs.get(page_format%(page)) 
    bid_response = bid_list.text.encode('utf8')
    #print bid_response

## 先拿一頁做試驗

In [26]:
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
page = 1 
bid_list = rs.get(page_format%(page)) 
bid_response = bid_list.text.encode('utf8') 
bid_soup = BeautifulSoup(bid_response) 
#print bid_response
bid_table = bid_soup.select('#print_area')[0]
#print bid_table

## 去掉標頭

In [27]:
bid_rows = bid_table.select('tr')[1:] 
#print bid_rows

## 去掉標頭跟頁次

In [28]:
bid_rows = bid_table.select('tr')[1:-1] 
#print bid_rows

## 索引範例

In [29]:
ary = [1,2,3,4,5,6] 
print ary[1:-1] 
#[2,3,4,5]

[2, 3, 4, 5]


## 抓出每一列所有的連結

In [31]:
bid_rows = bid_table.select('tr')[1:-1] 
for bid_row in bid_rows: 
    print bid_row.select('a')

    #print links


[<a href="../main/pms/tps/atm/atmAwardAction.do?newEdit=false&amp;searchMode=common&amp;method=inquiryForPublic&amp;pkAtmMain=51603142&amp;tenderCaseNo=104037">
<u>新竹郵局各單位暨所轄支局104年度40歲以上員工一般健康檢查</u>
</a>, <a href="../main/pms/tps/atm/atmAwardAction.do?newEdit=false&amp;searchMode=common&amp;method=inquiryForPublic&amp;pkAtmMain=51603142&amp;tenderCaseNo=104037">
<u>001</u>
</a>]
[<a href="../main/pms/tps/atm/atmAwardAction.do?newEdit=false&amp;searchMode=common&amp;method=inquiryForPublic&amp;pkAtmMain=51603189&amp;tenderCaseNo=1040202">
<u>104年第二次電腦軟體共同供應契約採購－雲端服務</u>
</a>, <a href="../main/pms/tps/atm/atmAwardAction.do?newEdit=false&amp;searchMode=common&amp;method=inquiryForPublic&amp;pkAtmMain=51603189&amp;tenderCaseNo=1040202">
<u>001</u>
</a>]
[<a href="../main/pms/tps/atm/atmAwardAction.do?newEdit=false&amp;searchMode=common&amp;method=inquiryForPublic&amp;pkAtmMain=51602392&amp;tenderCaseNo=GUA20150522-1">
<u>訪賓機票採購</u>
</a>, <a href="../main/pms/tps/atm/atmAwardAction.do?newEd

## 用[0]取第一個連結

In [32]:
for bid_row in bid_rows: 
    links =  bid_row.select('a')[0]['href']
    print links


../main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603142&tenderCaseNo=104037
../main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603189&tenderCaseNo=1040202
../main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51602392&tenderCaseNo=GUA20150522-1
../main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603176&tenderCaseNo=104W038
../main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603207&tenderCaseNo=1040071869
../main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603035&tenderCaseNo=YG10403
../main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51604288&tenderCaseNo=104812124A
../main/pms/tps/atm/atmAwardAction.do?newEdit=false

## 取得實際連結

In [33]:
for bid_row in bid_rows: 
    link = [tag['href'] for tag in bid_row.select('a')][0] 

    link_href = "http://web.pcc.gov.tw/tps" + link[2:] 

    #print link_href


## 使用urljoin

In [34]:
import urlparse
for bid_row in bid_rows: 
    link = [tag['href'] for tag in bid_row.select('a')][0] 

    link_href = urlparse.urljoin("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", link)

    #print link_href


## 複習一下如何使用Python做檔案寫入

In [35]:
f = open("test.txt", 'w') 
f.write("Hello World\n") 
f.close() 

## 將每頁標案清單的連結存入檔案

In [36]:
bid_file = open("bid_list.txt", 'w') 
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    bid_list = rs.get(page_format%(page)) 
    bid_response = bid_list.text.encode('utf8') 
    bid_soup = BeautifulSoup(bid_response) 
    bid_table = bid_soup.select("#print_area") [0]
    bid_rows = bid_table.select('tr')[1:-1] 
    for bid_row in bid_rows:         
        link = bid_row.select('a')[0]['href'] 
        link_href = "http://web.pcc.gov.tw/tps" + link[2:] 
        bid_file.write(link_href + "\n") 
bid_file.close() 

## 0. 引入該用的package

In [37]:
import requests
import urlparse 
from bs4 import BeautifulSoup 
from math import ceil

## 1. 設定查詢條件

In [42]:
payload = { 'method':'search', 'searchMethod':'true', 'searchTarget':'ATM',
 'orgName':'', 'orgId':'', 'hid_1':'1',
 'tenderName':'', 'tenderId':'', 'tenderStatus':'4,5,21,29',
 'tenderWay':'', 'awardAnnounceStartDate':'104/07/24',
 'awardAnnounceEndDate':'104/07/24', 'radProctrgCate':'3', 
 'proctrgCate':'3', 'tenderRange':'', 'minBudget':'',
 'maxBudget':'', 'item':'','hid_2':'1',
 'gottenVendorName':'', 'gottenVendorId':'', 'hid_3':'1',
 'submitVendorName':'', 'submitVendorId':'', 'location':'',
 'priorityCate':'', 'isReConstruct':'', 'btnQuery':'查詢' }

rs = requests.session() 
user_post = rs.post("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", data=payload) 
response_text = user_post.text.encode('utf8')


## 2. 計算抓取頁數

In [43]:
soup = BeautifulSoup(response_text) 
rec_number_element = soup.select(".T11b" )[0] 
rec_number = int(rec_number_element.text)
page_number = int(ceil(float(rec_number) / 100))


## 3. 依每頁抓取所有標案連結 &     4. 存入每頁標案連結

In [44]:
bid_file = open("bid_list.txt", 'w') 
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    bid_list = rs.get(page_format%(page)) 
    bid_response = bid_list.text.encode('utf8') 
    bid_soup = BeautifulSoup(bid_response)
    bid_table = bid_soup.select("#print_area")[0] 
    bid_rows = bid_table.select('tr')[1:-1] 
    for bid_row in bid_rows: 
        link = [tag['href'] for tag in bid_row.select('a')][0] 
        link_href = urlparse.urljoin("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", link) 
        bid_file.write(link_href + "\n") 
bid_file.close() 

## 計算總共存了多少個連結

In [45]:
line_num = 0 
f = open("bid_list.txt", "r") 
for line in f.readlines(): 
    line_num = line_num + 1 
print line_num 
f.close() 

214


## 判斷抓下來的資料筆數  是否與網頁上呈現的相同

In [46]:
soup = BeautifulSoup(response_text) 
rec_number_element = soup.select(".T11b" )[0] 
rec_number = int(rec_number_element.text)

line_num = 0 
f = open("bid_list.txt", "r") 
for line in f.readlines(): 
    line_num = line_num + 1 
print line_num 
f.close() 

#使用if 判斷
if rec_number == line_num: 
    print "record number on web page is equal to number of lines in file" 


214
record number on web page is equal to number of lines in file


## 使用requests.get 抓取標案細節

In [47]:
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603709&tenderCaseNo=104040") 
response = request_get.text.encode('utf8') 
#print response

## 取得id為printArea 的區塊

In [48]:
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603709&tenderCaseNo=104040") 
response = request_get.text.encode('utf8') 
soup = BeautifulSoup(response)
printarea = soup.select('#printArea') [0]
#print printarea


## 將標案資料寫進檔案中

In [49]:
bid_detail = open("bid_detail.txt", 'w') 
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603709&tenderCaseNo=104040") 
response = request_get.text.encode('utf8') 
soup = BeautifulSoup(response)
printarea = soup.select('#printArea') [0]
bid_detail.write(printarea) 
bid_detail.close()

TypeError: expected a character buffer object

## 使用prettify 將soup 抓到的tag   轉換成str

In [50]:
bid_detail = open("bid_detail.txt", 'w') 
request_get = requests.get("http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51603709&tenderCaseNo=104040") 
response = request_get.text.encode('utf8') 
soup = BeautifulSoup(response)
printarea = soup.select('#printArea') [0] 
bid_detail.write(printarea.prettify("utf-8")) 
bid_detail.close()

## 怎麼找出案號是否會重複?

In [51]:
case_dic = {} 
f = open("bid_list.txt", "r") 
for line in f.readlines(): 
    rec = line.strip() 
    caseno = rec.split('tenderCaseNo=')[1] 
    if caseno not in case_dic: 
        case_dic[caseno] = 1 
    else: 
        print caseno 
f.close()

1040707
1040702


## 如何從url 取得這兩個值?

In [52]:
link = "http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328" 

## 使用split 

In [53]:
rear_substring = link.split("pkAtmMain=")[1] 
param = rear_substring.split("&tenderCaseNo=") 
pkAtmMain= param[0] 
caseno= param[1] 
print pkAtmMain,caseno 

51239291 0607L1030328


## re.match 範例

In [54]:
import re 
m = re.match(r"(?P<FirstName>\w+) (?P<LastName>\w+)", "David Chiu") 
print m.group("FirstName"), m.group("LastName") 

David Chiu


## 使用re.match

In [55]:
import re 
link = "http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51239291&tenderCaseNo=0607L1030328" 
m = re.match(r"([^ ]+)pkAtmMain=(?P<pkAtmMain>\w+)&tenderCaseNo=(?P<tenderCaseNo>\w+)", link) 
print m.group('pkAtmMain'), m.group('tenderCaseNo')

51239291 0607L1030328


## 抓取標案細節完成版

In [ ]:
# -*- coding: utf-8 -*- 
from bs4 import BeautifulSoup 
import re, requests 
import time

bid_list = open('bid_list.txt', 'r') 
for line in bid_list.readlines(): 
    pagelink = line.strip() 
    m = re.match(r"([^ ]+)pkAtmMain=(?P<pkAtmMain>.+)&tenderCaseNo=(?P<tenderCaseNo>.+)", pagelink) 
    filename = "%s_%s"%(m.group('pkAtmMain'), m.group('tenderCaseNo')) 
    request_get = requests.get(pagelink) 
    response = request_get.text.encode('utf8') 
    soup = BeautifulSoup(response) 
    printarea = soup.select('#printArea')[0] 
    #print filename
    bid_detail = open("gov/%s.txt"%(filename), 'w') 
    bid_detail.write(printarea .prettify("utf-8")) 
    bid_detail.close() 
    time.sleep(0.01)
bid_list.close()


## Findbook 詢價

In [60]:
import requests
import time 
booklist = [9789863021582, 9789866031472]
find_url = 'http://findbook.tw/book/%s/basic'
for book in booklist:
    f = open('%s.html'%(book), 'w')
    res = requests.get(find_url%(book))
    f.write(res.text.encode('utf-8'))
    f.close()
    time.sleep(1)

## 取得頁面的encoding

In [61]:
import requests
import time 
res = requests.get('http://findbook.tw/book/9789863021582/basic')
print res.encoding

ISO-8859-1


## 更改response 的encoding

In [62]:
import requests
import time 
res = requests.get('http://findbook.tw/book/9789863021582/basic')
res.encoding= 'utf-8'
print res.encoding

utf-8


## 再度修改程式

In [63]:
import requests
import time 

booklist = ['9789863021582', '9789866031472']
find_url = 'http://findbook.tw/book/%s/basic'
for book in booklist:
    f = open('%s.html'%(book), 'w')
    res = requests.get(find_url%(book))
    res.encoding= 'utf-8'
    f.write(res.text.encode('utf-8'))
    f.close()
    time.sleep(1)

## 用爬蟲取得PTT資訊

In [64]:
import requests
res = requests.get('http://www.ptt.cc/bbs/Food/index.html')
print res.text


C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


SSLError: hostname 'www.ptt.cc' doesn't match either of 'images.ptt.cc', 'ptt.cc'

## 新增Verify 的選項

In [65]:
import requests
payload = {'from':'/bbs/sex/index.html','yes':'yes'}
rs = requests.session()
res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data= payload)
res2 = rs.get('http://www.ptt.cc/bbs/sex/index.html', verify=False)
#print res2.text

C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
C:\Python27\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Python27\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
C:\Python27\lib\site-packages\requests\packages\urllib3\

## 取得所有連結

In [71]:
import requests
from bs4 import BeautifulSoup
res = requests.get('http://www.ptt.cc/bbs/Food/index.html', verify=False)
soup = BeautifulSoup(res.text)
links = [tag['href'] for tag in soup.select('.r-ent a')] 
#print links

C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
C:\Python27\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


## 用爬蟲取得Yahoo股市的投資組合

In [91]:
# -*- coding: utf-8 -*- 
import requests
from bs4 import BeautifulSoup
rs = requests.session()
res = rs.get('https://login.yahoo.com/')
soup = BeautifulSoup(res.text)
login_form = soup.find('form', attrs={'id':'mbr-login-form'})
hiddens = login_form.findAll('input', attrs={'type':'hidden'})
payload = {}
for h in hiddens:
    payload[str(h.get('name'))] = str(h.get('value'))

payload['login'] = 'xpertslayers@yahoo.com.tw'
payload['passwd'] = 'Amadeus38'
print login_form.get('action')
post_url = str('https://login.yahoo.com/config/login?.intl=tw&.done=https%3A%2F%2Ftw.yahoo.com%2F')
res2 = rs.post(post_url, cookies=res.cookies, data=payload)


C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


/


## 轉跳到投資組合頁面

In [93]:
res3 = rs.get('https://tw.stock.yahoo.com/pf/mypf', cookies=res2.cookies)
#print res3.text.encode('utf-8')


C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


## 取得Facebook的朋友清單

In [95]:
import requests, json
access_token = 'CAACEdEose0cBAOIPDjeyJZBhK0Daj9vmuZCnSju0xV73CerTVNZAxKxP8ZB9z1BJDNZAOFUZA8Pxr5HE6KgEPjryFQGU3k48jNaEpQiBc0E0D7rrHAxwLFIC3Yi8k8WfkZBA5NflHBh39f5EY1g9ftbrtz40AHLR3EuzUQqFgBAGZBCat9dZAMDbrwa26NKzXtnkK1C3p9AqDtHQL32cHiEFWiqkyCWtPW0gZD'
graph_url = 'https://graph.facebook.com/v2.0/me/friends?access_token=%s'
res = requests.get(graph_url%(access_token))
j = json.loads(res.text)
#for i in j['data']:
#    print i['name']

C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [96]:
import requests
from bs4 import BeautifulSoup
rs = requests.get('https://www.ptt.cc/bbs/Stock/index.html', verify=False)
soup = BeautifulSoup(rs.text)
for ent in  soup.select('.r-ent'):
    print ent.select('.title')[0].text.strip(), ent.select('.date')[0].text

(本文已被刪除) [sunandy]  8/29
[請益] 股票無預警大漲  8/29
[新聞] 台股收復8000點 禁空令將解？  8/29
[新聞] 日月光：公開收購一定會執行到底  8/29
[新聞] 「紅色供應鏈」重擊 出口將連7黑  8/29
[心得] 貨幣和原物料異常波動  8/29
[公告] 股票板板規     (201411修) 11/04
[公告] 精華區導覽Q&A  (201501版)  1/25
[公告] 發文注意事項  4/17
Re: [其他] 協尋阿鼻的受害者  8/25
[閒聊]2015/08/28 盤後閒聊  8/28


C:\Python27\lib\site-packages\requests\packages\urllib3\util\ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
C:\Python27\lib\site-packages\requests\packages\urllib3\connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
